<a href="https://colab.research.google.com/github/meti-94/TextGeneration/blob/main/personachat_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar -xf  simple-examples.tgz
!mkdir data
!mv /content/simple-examples/data/ptb.train.txt data/
!mv /content/simple-examples/data/ptb.valid.txt data/
!mv /content/simple-examples/data/ptb.test.txt data/
!rm -rf ./simple_examples

In [1]:
!pip install "ray[tune]" transformers datasets==1.1.2
# !pip install transformers
!ray install-nightly
!pip install rouge_score
!pip install -U ray
# !pip install -U transformers datasets
!pip install tensorboardX


     |████████████████████████████████| 47.5MB 63kB/s 
     |████████████████████████████████| 2.1MB 38.0MB/s 
     |████████████████████████████████| 153kB 36.3MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 133kB 44.5MB/s 
     |████████████████████████████████| 204kB 39.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 1.3MB 33.3MB/s 
     |████████████████████████████████| 3.1MB 37.7MB/s 
     |████████████████████████████████| 122kB 42.5MB/s 
     |████████████████████████████████| 3.3MB 36.4MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 
     |████████████████████████████████| 245kB 41.5MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 296kB 39.4MB/s 
     |████████████████████████████████| 143kB 23.6MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-

In [6]:
import transformers
transformers.__version__

'4.5.1'

In [8]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
!mkdir models

--2021-04-21 13:34:49--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  35.5MB/s    in 6.3s    

2021-04-21 13:34:56 (31.8 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# import ray
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import AsyncHyperBandScheduler



from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset
from transformers import AdamW
from tqdm import tqdm
import json
import random 
from sklearn.model_selection import train_test_split
from datetime import datetime
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from tqdm import tqdm_notebook as tqdm
import logging
import datasets
logging.basicConfig(level=logging.DEBUG)
from customize.trainer_customization import CustomSeq2SeqTrainer

In [5]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<persona>", "<speaker1>", "<speaker2>", "<pad>"]

ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', '<persona>']}

encoder_max_length=512
decoder_max_length=128

def read_data(data_json_path='/content/personachat_self_original.json'):
  with open(data_json_path) as json_file:
    data_dict = json.load(json_file)
  return data_dict

def data_to_samples(data_dict, test=False):
  samples=[]
  for dialogue in (data_dict['train'] if test==False else data_dict['valid']):
    original_persona = dialogue['personality']
    
    for item in dialogue['utterances']:
      original_persona = [original_persona[-1]] + original_persona[:-1]
      history = item['history']
      response = item['candidates'][-1]
      samples.append({
          'persona':original_persona,
          'history':history,
          'response':response
      })
  return samples
  
def bertified(samples):
  bertified_data = []
  for item in samples:
    persona = ' <persona> '.join(item['persona'])
    persona = '<bos> ' + persona
    history = ''
    speakers = [" <speaker1> ", " <speaker2> "]
    speaker = 0
    for hst in item['history'][::-1]:
      history = speakers[speaker] + hst + history
      speaker = 1 - speaker
    response = '<speaker2> ' + item['response'] + ' <eos>'
    bertified_data.append({
          'persona':persona.replace('  ', ' '),
          'history':history.replace('  ', ' '),
          'input': persona.replace('  ', ' ')+' '+history.replace('  ', ' '), 
          'response':response.replace('  ', ' ')
      })
  return bertified_data

def bertified_to_model_food(bertified_data, tokenizer):
    model_food = []
    pbar = tqdm(bertified_data)
    pbar.set_description('Tokenizing Data Sample ...')
    for item in pbar:
        inputs = tokenizer(item['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
        outputs = tokenizer(item['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
        _item = {}
        _item["input_ids"] = inputs.input_ids
        _item["attention_mask"] = inputs.attention_mask
        _item["decoder_input_ids"] = outputs.input_ids
        _item["decoder_attention_mask"] = outputs.attention_mask
        _item["labels"] = outputs.input_ids.copy()
        model_food.append(_item)
    return model_food 

In [7]:
class PersonaDataset_v3(Dataset):
  '''
      Convert Data to proper Tensor dataset
  '''
  def __init__(self, samples):
    self.samples = samples
    self.n_samples = len(self.samples)

  def __getitem__(self, index):
    # returns specific item
    return self.samples[index]

  def __len__(self):
    return self.n_samples

In [8]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("prajjwal1/bert-tiny", "prajjwal1/bert-tiny")
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

data = read_data('/home/textmining/HDD/mehdi_jafari/git/personachat_self_original.json')
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data, test_size=0.15, random_state=99)
train = bertified_to_model_food(train, tokenizer)
valid = bertified_to_model_food(valid, tokenizer)
test_data = read_data('/home/textmining/HDD/mehdi_jafari/git/personachat_self_original.json')
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)
test = bertified_to_model_food(test, tokenizer)



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationsh

In [9]:
# train_dataset = PersonaDataset_v3(list(zip(*map(one_line_tokenizer, tqdm(train)))))
train_dataset = PersonaDataset_v3(train)
valid_dataset = PersonaDataset_v3(valid)
test_dataset = PersonaDataset_v3(test)



In [10]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/dataset_infos.json HTTP/1.1" 404 0
DEBUG:filelock:Attempting to acquire lock 139955032433488 on /home/textmining/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 139955032433488 acquired on /home/tex

In [11]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# model.config.max_length = 142
# model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [43]:
# from torch import nn
# from transformers.trainer_utils import PredictionOutput
# import random 
# class CustomSeq2SeqTrainer(Seq2SeqTrainer):
#     def evaluate(
#         self,
#         eval_dataset: Optional[Dataset] = None,
#         ignore_keys: Optional[List[str]] = None,
#         metric_key_prefix: str = "eval",
#         max_length: Optional[int] = 128,
#         num_beams: Optional[int] = None,
#     ) -> Dict[str, float]:
#         return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
#     def prediction_step(
#         self,
#         model: nn.Module,
#         inputs: Dict[str, Union[torch.Tensor, Any]],
#         prediction_loss_only: bool,
#         ignore_keys: Optional[List[str]] = None,
#     ) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
        
#         (loss, logits, labels) = super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)
#         random_subset = random.sample(range(0, len(inputs['labels'])), 10)
#         for item in random_subset:
#             persona_history_input = self.tokenizer.batch_decode([inputs['input_ids'][item]])
#             real_output = self.tokenizer.batch_decode([inputs['labels'][item]])
#             model_output = self.tokenizer.batch_decode([logits[item]])
#             print("History & Persona & Input: ", persona_history_input[0].replace("<pad>", ""))
#             print("Input: ", real_output[0].replace("<pad>", ""))
#             print("Output: ", model_output[0].replace("<pad>", ""))
#         return loss, logits, labels
        

In [12]:
steps = 1000
batch_size = 64
training_args = Seq2SeqTrainingArguments(
    output_dir="./models/",
    do_eval=True,
    do_predict =True,

    
    evaluation_strategy="steps",
    save_strategy = 'steps',
    
    eval_steps=steps,
    logging_steps=steps,
    save_steps=steps,
    
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    fp16=True, 
    
    
    save_total_limit = 2,
)

# instantiate trainer
def model_init():
    return model


trainer = CustomSeq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
#     test_dataset=test_dataset,
    tokenizer=tokenizer,
#     max_length = 128
)




In [ ]:
trainer.train()

/home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss


In [39]:
loss, logits, labels = trainer.predict(test_dataset)

History & Persona & Input:  [CLS] <bos> i have a big library at home. <persona> i'm a museum tour guide. <persona> i'm partly deaf. <persona> i love to drink fancy tea. <speaker1> hi how are you doing? i am okay how about you? <speaker2> i am doing good. just sipping tea. what do you do for work? <speaker1> i used to do home health aide but now i am disabled. <speaker2> i am sorry to hear that. what happened <speaker1> i graduated and got my license worked a while and became i'll. <speaker2> so what do you do now for fun? i like to read. <speaker1> i have dogs and i walk them. and a cat. <speaker2> what kind of dogs do you have? <speaker1> i enjoy reading read about michael jackson and psychology today. <speaker2> i love reading. i have a big library at my house. <speaker1> that's very nice i love to read to and read about journals. <speaker2> i've a tough time hearing so reading is my happy place. <speaker1> cool that is okay. my mom was very tough on me. [SEP]                        

History & Persona & Input:  [CLS] <bos> whats up i am a 46 year old single dad 46 a kids. <persona> i live in california but the recording artist market was dead last year. <persona> i prefer being inside. <persona> i'm a recording engineer and own my own studio. <persona> my ex cheated and left me for a lawyer. <speaker1> hi, kevin here. i love mexican food, aerosmith and nascar racing and a construction worker. <speaker2> i'm a recording engineer myself. currently living in california <speaker1> great! i wear a yellow construction worker hat to aerosmith concerts. <speaker2> that sounds fun. i'd rather be inside though. <speaker1> wow! what artists you work with? i started working at sixteen as a construction model. [SEP]                                                                                                                                                                                                                                                                           

DEBUG:filelock:Attempting to acquire lock 139835798107024 on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139835798107024 acquired on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to release lock 139835798107024 on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139835798107024 released on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to acquire lock 139836002998160 on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139836002998160 acquired on /home/textmining/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:/home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/datasets/metric.py:Removing /home/textmining/.cache

In [96]:
print(type(loss), type(loss), type(loss))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [97]:
print(loss)

[[  101 30526  1045 ... 30524 30524 30524]
 [  101 30526  2008 ... 30524 30524 30524]
 [  101 30526  2008 ... 30524 30524 30524]
 ...
 [  101 30526  2008 ... 30524 30524 30524]
 [  101 30526  1045 ... 30524 30524 30524]
 [  101 30526  1045 ... 30524 30524 30524]]


In [90]:
b.shape

(200, 128)

In [40]:
# from ray import tune
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import AsyncHyperBandScheduler

trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=4, # number of trials
)

== Status ==
Memory usage on this node: 11.0/31.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/2 GPUs, 0.0/15.6 GiB heap, 0.0/7.8 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /home/textmining/ray_results/_objective_2021-04-23_16-10-57
Number of trials: 4/4 (3 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _objective_c4b35_00000 | RUNNING  |       |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_c4b35_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 |  7.08379 |
| _objective_c4b35_00002 | PENDING  |

(pid=17964) /home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(pid=17964)   "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>


(pid=18007) /home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(pid=18007)   "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <

...

(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <

(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <

2021-04-23 17:00:15,347	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <

KeyboardInterrupt: 